In [103]:
%load_ext autoreload
%autoreload 2

import os
import sys
os.environ['UTILS_PATH'] = r'C:\Users\Mi\Desktop\code\git\python-utils-ak'
os.environ['PROJECT_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat' 
sys.path.append(os.environ['PROJECT_PATH'])
from app.interactive_imports import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [153]:
request = anyconfig.load(r"C:\Users\Mi\YandexDisk\Shared\2020 umalat\schedules\2020.12.04.json")

values = []
for boiling in request['Boilings']:
    for sku_id, sku_kg in boiling['SKUVolumes'].items():
        values.append([boiling['BoilingId'], sku_id, sku_kg])
df = pd.DataFrame(values)

values = []
for boiling_id, boiling_grp in df.groupby(0):
    boiling_dic = boiling_grp[[1, 2]].set_index(1).to_dict(orient='index')
    boiling_dic = {k: v[2] for k, v in boiling_dic.items()}  # {'34': 1110.0, '35': 0.0, ...}
    boiling_dic = {cast_sku(k): v for k, v in boiling_dic.items()} # {<SKU 34>: 1110.0, ...}
    total_kg = sum(boiling_dic.values())

    # round to get full
    # todo: proper logic
    boiling_type = 'salt' if str(cast_boiling(boiling_id).percent) == '2.7' else 'water'
    volume = 1000 if boiling_type == 'water' else 850
    
    total_kg = custom_round(total_kg, volume, rounding='floor')
    
        
    n_boilings = int(total_kg / volume)
    for i in range(n_boilings):
        cur_kg = volume

        boiling_request = OrderedDict()
        for k, v in list(boiling_dic.items()):
            boil_kg = min(cur_kg, boiling_dic[k])

            boiling_dic[k] -= boil_kg
            cur_kg -= boil_kg

            if k not in boiling_request:
                boiling_request[k] = 0
            boiling_request[k] += boil_kg

            if cur_kg == 0:
                break

        if cur_kg != 0:
            # any non-zero
            print('Non-zero')
            k = [k for k, v in boiling_request.items() if v != 0][0]
            boiling_request[k] += cur_kg
        
        # convert to list
        boiling_request = [[k, v] for k, v in boiling_request.items() if v != 0]
        values.append([boiling_id, boiling_request])
                           
df = pd.DataFrame(values, columns=['boiling_id', 'boiling_request'])
df['boiling_id'] = df['boiling_id'].astype(str)
df['boiling_type'] = df['boiling_id'].apply(lambda boiling_id: 'salt' if str(cast_boiling(boiling_id).percent) == '2.7' else 'water')
df['used'] = False

df['boiling_label'] = df['boiling_id'].apply(lambda boiling_id: '{} {}{}'.format(cast_boiling(boiling_id).percent, 
                                             cast_boiling(boiling_id).ferment, 
                                             '' if cast_boiling(boiling_id).is_lactose else ' без лактозы'))

df['volume'] = df['boiling_request'].apply(lambda req: sum([v for k, v in req]))

df['boiling_request'] = df['boiling_request'].apply(lambda req:  [[cast_sku(k), round(v)] for k, v in req if v != 0])

values = []
for i, row in df.iterrows():
    for sku, kg in row['boiling_request']:
        values.append([i + 1, cast_boiling(row['boiling_id']), sku, kg])

boiling_plan_df = pd.DataFrame(values, columns=['id', 'boiling', 'sku', 'kg'])
boiling_plan_df.head()

,id,boiling,sku,kg
0,1,<Boiling 1>,<SKU 14>,800
1,1,<Boiling 1>,<SKU 34>,50
2,2,<Boiling 1>,<SKU 34>,650
3,2,<Boiling 1>,<SKU 39>,200
4,3,<Boiling 1>,<SKU 39>,850


In [184]:
df = pd.DataFrame(cast_dict(r"C:\Users\Mi\YandexDisk\Shared\2020 umalat\schedules\2020.12.04 request_by_boilings.json"))

df['boiling_request'] = df['boiling_request'].apply(lambda req:  [[cast_sku(k), v] for k, v in req if v != 0])

for i, row in df.iterrows():
    df.at[i, 'boiling_request'] = [[k,  round(v * row['volume'] / sum([v for k, v in row['boiling_request']]))] for k, v in row['boiling_request']]
df['used'] = False


values = []
for i, row in df.iterrows():
    for sku, kg in row['boiling_request']:
        values.append([i + 1, sku.boilings[0], sku, kg])
boiling_plan_df = pd.DataFrame(values, columns=['id', 'boiling', 'sku', 'kg'])
boiling_plan_df.head()

,id,boiling,sku,kg
0,1,<Boiling 2>,<SKU 1>,850
1,2,<Boiling 1>,<SKU 34>,850
2,3,<Boiling 1>,<SKU 14>,283
3,3,<Boiling 1>,<SKU 4>,567
4,4,<Boiling 1>,<SKU 4>,850


In [185]:
values = db.session.query(SKU).all()
skus = pd.DataFrame(values, columns=['sku'])
skus['boiling'] = skus['sku'].apply(lambda sku: sku.boilings[0])
skus['id'] = ''
skus['kg'] = 0
skus = skus[['id', 'boiling', 'sku', 'kg']]
skus.head()

,id,boiling,sku,kg
0,,<Boiling 2>,<SKU 1>,0
1,,<Boiling 2>,<SKU 2>,0
2,,<Boiling 1>,<SKU 3>,0
3,,<Boiling 1>,<SKU 4>,0
4,,<Boiling 1>,<SKU 5>,0


In [164]:
def gen_constructor_df(boiling_plan_df):
    # create dataframe for samples
    df = boiling_plan_df.copy()
    df['name'] = df['sku'].apply(lambda sku: sku.name)
    # todo: make properly
    df['boiling_name'] = df['boiling'].apply(lambda b: '{} {} {}'.format(b.percent, b.ferment, '' if b.is_lactose else 'без лактозы'))
    df['boiling_volume'] = np.where(df['boiling_name'].str.contains('2.7'), 850, 1000)
    df['form_factor'] = df['sku'].apply(lambda sku: sku.form_factor.name) 
    df['boiling_id'] = df['boiling'].apply(lambda b: b.id)
    df = df[['id', 'boiling_id', 'boiling_name','boiling_volume','form_factor', 'name', 'kg']]
    df = df.sort_values(by=['boiling_id', 'id', 'boiling_name', 'form_factor', 'name'])
    return df.reset_index(drop=True)
gen_constructor_df(skus).head()

,id,boiling_id,boiling_name,boiling_volume,form_factor,name,kg
0,,1,2.7 Альче,850,Для пиццы,"Моцарелла ""Unagrande"", 45%, 0,12 кг, ф/п (кубики)",0
1,,1,2.7 Альче,850,Для пиццы,"Моцарелла ""Unagrande"", 45%, 3 кг, пл/л",0
2,,1,2.7 Альче,850,Для пиццы,"Моцарелла для пиццы ""Unagrande"", 45%, 0,46 кг, в/у, (8 шт)",0
3,,1,2.7 Альче,850,Для пиццы,"Моцарелла для сэндвичей ""Unagrande"", 45%, 0,28 кг, т/ф, (8 шт)",0
4,,1,2.7 Альче,850,Для пиццы,"Моцарелла палочки ""Unagrande"", 45%, 0,12 кг, т/ф",0


# Draw all skus for copy-paste

In [171]:
wb, sheet = init_empty_sheet()

for i, v in enumerate([15, 15, 15, 15, 15, 50, 15], 1):
    sheet.column_dimensions[get_column_letter(i)].width = v 

cur_i = 1
draw_row(sheet, cur_i, ['id варки', "Номер варки", 'Тип варки', 'Объем варки', 'Форм фактор', 'SKU', 'КГ', 'Разделитель'], font_size=8)
cur_i += 1

df = gen_constructor_df(boiling_plan_df)

values = []
for id, grp in df.groupby('id'):
    for i, row in grp.iterrows():
        v = []
        v += list(row.values)
        v += ['']
        values.append(v)
        
    # add separator
    values.append(['-'] * (len(df.columns) + 1))
    
for v in values:
    draw_row(sheet, cur_i, v, font_size=8)
    cur_i += 1

cur_i += 1
df = gen_constructor_df(skus)
values = []
for boiling_name, grp1 in df.groupby('boiling_name'):
    for form_factor, grp2 in grp1.groupby('form_factor'):
        for i, row in grp2.iterrows():
            values.append(list(row.values))

for v in values:
    draw_row(sheet, cur_i, v, font_size=8)
    cur_i += 1
    
wb.save('constructor.xlsx')

# Parse Constructor

In [172]:
import openpyxl

In [187]:
wb = openpyxl.load_workbook(filename=r"C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\research\akadaner\2020.12.13 constructor_sample.xlsx", data_only=True)
ws = wb['Sheet']
values = []
for i in range(1, 200):
    if ws.cell(i, 1).value:
        values.append([ws.cell(i, j).value for j in [1, 2, 6, 7]])

In [192]:
df = pd.DataFrame(values[1:], columns=['boiling', 'id', 'sku', 'kg']) # first value is header
df = df[df['boiling'] != '-']
df = df[~df['kg'].isnull()]
df = df[['id', 'boiling', 'sku', 'kg']] # reorder
df.head()

,id,boiling,sku,kg
0,1.0,2,"Моцарелла ""Pretto"" (для бутербродов), 45%, 0,2 кг, т/ф, (9 шт)",850
2,2.0,1,"Сулугуни ""Умалат"", 45%, 0,37 кг, т/ф, (6 шт)",850
4,3.0,1,"Моцарелла ""Unagrande"", 45%, 3 кг, пл/л",567
5,3.0,1,"Моцарелла для пиццы ""Unagrande"", 45%, 0,46 кг, в/у, (8 шт)",283
7,4.0,1,"Моцарелла ""Unagrande"", 45%, 3 кг, пл/л",850


In [193]:
df.to_csv('2020.12.13 boiling_plan.csv', index=None)